In [2]:
import pandas as pd
import altair as alt

dir = '../Data'

weather = pd.read_csv(f'{dir}/weather_clean.csv')
collisions = pd.read_csv(f'{dir}/collisions_clean.csv')
weather_aggregated = pd.read_csv(f'{dir}/weather_aggregated.csv')

alt.data_transformers.enable("vegafusion")
# select the year from the CRASH DATE column
collisions['YEAR'] = collisions['CRASH DATE'].astype(str).str[:4]

In [3]:
weather_aggregated['DATE'] = pd.to_datetime(weather_aggregated['DATE']).dt.strftime('%Y-%m-%d')
weather_aggregated.head()

,DATE,AWND,PRCP,TMAX,TMIN,TOBS,BEAUFORT_SCALE,PRCP_SCALE,MEAN_TEMP
0,2018-06-01,1.6250,4.354762,24.921429,17.371429,18.683333,Light Breeze,Slight,20.325397
1,2018-06-02,3.4875,5.296296,29.330769,18.800000,21.825000,Gentle Breeze,Slight,23.318590
2,2018-06-03,5.0000,6.153165,23.800000,12.353846,15.125000,Gentle Breeze,Slight,17.092949
3,2018-06-04,2.8875,13.223256,20.700000,10.392308,10.750000,Light Breeze,Slight,13.947436
4,2018-06-05,3.4875,0.801266,23.092857,11.500000,13.233333,Gentle Breeze,Slight,15.942063


## Is there a correlation between weather conditions and accidents?

In [4]:
collisions['CRASH DATE'] = pd.to_datetime(collisions['CRASH DATE']).dt.strftime('%Y-%m-%d')

collisions_by_day = collisions[['CRASH DATE', 'COLLISION_ID']].groupby(['CRASH DATE']).count().reset_index()
collisions_by_day.head()

,CRASH DATE,COLLISION_ID
0,2018-06-01,751
1,2018-06-02,622
2,2018-06-03,525
3,2018-06-04,698
4,2018-06-05,688


In [5]:
merged_data = pd.merge(weather_aggregated, collisions_by_day, left_on='DATE', right_on='CRASH DATE')
merged_data.drop(columns=['CRASH DATE'], inplace=True)
merged_data.rename(columns={'COLLISION_ID': 'COLLISION COUNT'}, inplace=True)
merged_data.head()

,DATE,AWND,PRCP,TMAX,TMIN,TOBS,BEAUFORT_SCALE,PRCP_SCALE,MEAN_TEMP,COLLISION COUNT
0,2018-06-01,1.6250,4.354762,24.921429,17.371429,18.683333,Light Breeze,Slight,20.325397,751
1,2018-06-02,3.4875,5.296296,29.330769,18.800000,21.825000,Gentle Breeze,Slight,23.318590,622
2,2018-06-03,5.0000,6.153165,23.800000,12.353846,15.125000,Gentle Breeze,Slight,17.092949,525
3,2018-06-04,2.8875,13.223256,20.700000,10.392308,10.750000,Light Breeze,Slight,13.947436,698
4,2018-06-05,3.4875,0.801266,23.092857,11.500000,13.233333,Gentle Breeze,Slight,15.942063,688


In [6]:
merged_data.describe()

,AWND,PRCP,TMAX,TMIN,TOBS,MEAN_TEMP,COLLISION COUNT
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000
mean,3.100534,4.261346,27.988266,18.638808,19.700600,22.109225,474.344262
std,0.911683,7.272843,3.793126,3.675226,3.653244,3.549131,189.606079
min,1.475000,0.000000,17.392308,6.785714,7.700000,11.334921,196.000000
25%,2.437500,0.019747,25.476786,16.659821,17.312500,20.287056,300.000000
50%,2.966071,0.897468,28.259066,19.194048,20.339286,22.611111,428.500000
75%,3.587500,5.289804,30.973626,21.427610,22.351429,24.767262,657.250000
max,6.162500,49.176531,35.366667,25.435714,26.766667,29.055556,845.000000


In [7]:
merged_data['BEAUFORT_SCALE'] = merged_data['BEAUFORT_SCALE'].astype('category')
merged_data['PRCP_SCALE'] = merged_data['PRCP_SCALE'].astype('category')

# min temperature is 11, max temperature is 29
bin_edges = list(range(10, 30, 2))
merged_data['TEMP_SCALE'] = pd.cut(merged_data['MEAN_TEMP'], bins=bin_edges, right=False)

In [8]:
merged_data['CASES_COUNT'] = 1

In [9]:
agg_data = merged_data.groupby(['BEAUFORT_SCALE', 'TEMP_SCALE']).agg({'COLLISION COUNT': 'sum', 'CASES_COUNT': 'sum'}).reset_index()

# change teh column type to categorical
agg_data['BEAUFORT_SCALE'] = agg_data['BEAUFORT_SCALE'].astype('category')
agg_data['TEMP_SCALE'] = agg_data['TEMP_SCALE'].astype('str')

agg_data['NORMALIZED_COLLISION_COUNT'] = agg_data['COLLISION COUNT'] / agg_data['CASES_COUNT']

agg_data.head()

C:\Users\jpniz\AppData\Local\Temp\ipykernel_19628\2379143315.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agg_data = merged_data.groupby(['BEAUFORT_SCALE', 'TEMP_SCALE']).agg({'COLLISION COUNT': 'sum', 'CASES_COUNT': 'sum'}).reset_index()


,BEAUFORT_SCALE,TEMP_SCALE,COLLISION COUNT,CASES_COUNT,NORMALIZED_COLLISION_COUNT
0,Gentle Breeze,"[10, 12)",307,1,307.000000
1,Gentle Breeze,"[12, 14)",801,3,267.000000
2,Gentle Breeze,"[14, 16)",2844,5,568.800000
3,Gentle Breeze,"[16, 18)",1581,3,527.000000
4,Gentle Breeze,"[18, 20)",6193,13,476.384615


In [10]:
# fill nan of NORMALIZED_COLLISION_COUNT with 0
agg_data['NORMALIZED_COLLISION_COUNT'].fillna(0, inplace=True)

In [11]:
alt.Chart(agg_data).mark_rect().encode(
    x=alt.X('TEMP_SCALE:O',
            title='Temperature Scale',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('BEAUFORT_SCALE:N',
            
            title='Beaufort Scale'),
    color=alt.Color('NORMALIZED_COLLISION_COUNT:Q', title='Collision Count')
).properties(
    width=500,
    height=300,
    title='Collision Count by Beaufort Scale and Precipitation Scale'
)

alt.Chart(...)

In [15]:
merged_data.columns

Index(['DATE', 'AWND', 'PRCP', 'TMAX', 'TMIN', 'TOBS', 'BEAUFORT_SCALE',
       'PRCP_SCALE', 'MEAN_TEMP', 'COLLISION COUNT', 'TEMP_SCALE',
       'CASES_COUNT'],
      dtype='object')

In [36]:
t1 = alt.Chart(merged_data).mark_point(
    filled=True,
    size=100,
    opacity=0.5
).encode(
    x=alt.X('MEAN_TEMP:Q',
            title='Mean Temperature',
            scale=alt.Scale(domain=[10, 30])),
    y=alt.Y('COLLISION COUNT:Q',
            title='Number of Collisions'),
    color=alt.Color('year(DATE):N',
                    title='Year')
).properties(
    title='Number of Collisions by Mean Temperature'
)

t2 = alt.Chart(merged_data).mark_point(
    filled=True,
    size=100,
    opacity=0.5
).encode(
    x=alt.X('PRCP:Q',
            title='Precipitation'),
    y=alt.Y('COLLISION COUNT:Q',
            title='Number of Collisions'),
    color=alt.Color('year(DATE):N',
                    title='Year'),
).properties(
    title='Number of Collisions by Precipitation'
)

t3 = alt.Chart(merged_data).mark_point(
    filled=True,
    size=100,
    opacity=0.5
).encode(
    x=alt.X('AWND:Q',
            scale=alt.Scale(domain=[1, 6.5]),
            title='Wind Speed'),
    y=alt.Y('COLLISION COUNT:Q',
            title='Number of Collisions'),
    color=alt.Color('year(DATE):N',
                    title='Year'),
).properties(
    title='Number of Collisions by Wind Speed'
)

t1 | t2 | t3

alt.HConcatChart(...)